# Quantum circuit examples generator

In [ ]:
import numpy as np
from qibo import models, gates, hamiltonians, callbacks
from qibo.models import Circuit
from qibo.symbols import X, Y, Z, I
import igraph as ig
import matplotlib.pyplot as plt
import random

## Random circuit

Generates a quantum circuit from a random graph where
- Number of edges >> number of Controlled - Z (CZ) gates
- Number of vertices >> number of qubits
- Graph generated from Erdos_Renyi (can be changed)
- `m` = number of CZ gates or `p` = probability of adding an edge between two arbitrary vertices
- `gate_max` = maximal number of random single-qubit gates (from a pull) that will be inserted between the two-qubit gates

In [ ]:
def random_circuit(qubits, gate_max, num_cz, p):
    if p == None:
        graph = ig.Graph.Erdos_Renyi(n=qubits, m=num_cz, directed=False, loops=False)
    elif num_cz == None:
        graph = ig.Graph.Erdos_Renyi(n=qubits, p=p, directed=False, loops=False)
    else:
        print(
            "Error: only the number of edges or the probability of adding an edge must be specified"
        )

    # adj_mat=graph.get_adjacency()
    # fig, ax = plt.subplots()
    # ig.plot(graph, target=ax, vertex_label=range(qubits))
    # graph.degree()

    edge_list = graph.get_edgelist()

    gates_pull = [gates.X, gates.Y, gates.Z, gates.H]  # pull of single-qubit gates

    circuit = models.Circuit(qubits)
    for i in range(len(edge_list)):
        rand_tmp = random.randint(
            0, gate_max
        )  # number of single-qubit gates between CZ
        for j in range(rand_tmp):
            sel_gate = random.choice(gates_pull)  # gate selected from the pull
            sel_qubit = random.randint(
                0, qubits - 1
            )  # qubit selected to apply the gate
            circuit.add(sel_gate(sel_qubit))

        circuit.add(
            gates.CZ(edge_list[i][0], edge_list[i][1])
        )  # 2-qubit gate from graph

    return circuit

In [ ]:
qubits = 5
num_cz = 10  # total number of 2 qubit gates
p = None  # random.uniform(0,1) # probability of adding an edge between two arbitrary edges
gate_max = 7  # max num of single-qubit gates between 2-qubit gates
rand_qc = random_circuit(qubits, gate_max, num_cz, p)
print(rand_qc.draw())

## Trotter expansion Hamiltonian simulation (in construction)

In [ ]:
import tequila as tq

In [ ]:
def HIsing(num_qubits, lam):
    # PBC
    ham = []
    coef = []
    ham.append(tq.paulis.Z(num_qubits - 1) * tq.paulis.Z(0))
    coef.append(1.0)
    ham.append(tq.paulis.X(num_qubits - 1))
    coef.append(lam)
    for i in range(num_qubits - 1):
        ham.append(tq.paulis.Z(i) * tq.paulis.Z(i + 1))
        coef.append(1.0)
        ham.append(tq.paulis.X(i))
        coef.append(lam)
    return (ham, coef)


def HXXZ(num_qubits, delta):
    # PBC
    ham = []
    coef = []
    ham.append(tq.paulis.Z(num_qubits - 1) * tq.paulis.Z(0))
    coef.append(delta)
    ham.append(tq.paulis.X(num_qubits - 1) * tq.paulis.X(0))
    coef.append(1.0)
    ham.append(tq.paulis.Y(num_qubits - 1) * tq.paulis.Y(0))
    coef.append(1.0)
    for i in range(num_qubits - 1):
        ham.append(tq.paulis.Z(i) * tq.paulis.Z(i + 1))
        coef.append(delta)
        ham.append(tq.paulis.X(i) * tq.paulis.X(i + 1))
        coef.append(1.0)
        ham.append(tq.paulis.Y(i) * tq.paulis.Y(i + 1))
        coef.append(1.0)
    return (ham, coef)


def HXXZ(num_qubits, delta):
    # split between even and odd sites
    ham_even = delta * tq.paulis.Z(0) * tq.paulis.Z(1)
    ham_even += tq.paulis.X(0) * tq.paulis.X(1)
    ham_even += tq.paulis.Y(0) * tq.paulis.Y(1)
    ham_odd = delta * tq.paulis.Z(num_qubits - 1) * tq.paulis.Z(0)
    ham_odd += tq.paulis.X(num_qubits - 1) * tq.paulis.X(0)
    ham_odd += tq.paulis.Y(num_qubits - 1) * tq.paulis.Y(0)
    for i in range(2, num_qubits - 1, 2):
        print(i)
        ham_even += delta * tq.paulis.Z(i) * tq.paulis.Z(i + 1)
        ham_even += tq.paulis.X(i) * tq.paulis.X(i + 1)
        ham_even += tq.paulis.Y(i) * tq.paulis.Y(i + 1)
        ham_odd += delta * tq.paulis.Z(i - 1) * tq.paulis.Z(i)
        ham_odd += tq.paulis.X(i - 1) * tq.paulis.X(i)
        ham_odd += tq.paulis.Y(i - 1) * tq.paulis.Y(i)
    return (ham_even, ham_odd)

In [ ]:
generator1 = tq.paulis.Y(0)
generator2 = tq.paulis.X(0)
circuit = tq.gates.Trotterized(
    generators=[generator1, generator2], angles=[1.0, 1.0], steps=1
)
tq.draw(circuit)

In [ ]:
generator3 = tq.paulis.Y(0) + tq.paulis.X(1)
circuit = tq.gates.Trotterized(generators=[generator3], angles=[1.0], steps=1)
tq.draw(circuit)

In [ ]:
num_qubits = 4
param = 0.3
ham, coef = HIsing(num_qubits, param)

circuit = tq.gates.Trotterized(generators=[ham], angles=coef, steps=1)
circuit += tq.gates.Trotterized(generators=[ham], angles=coef, steps=1)
tq.draw(circuit)

openqasmcode = tq.export_open_qasm(circuit)

c = models.Circuit.from_qasm(openqasmcode)

print(c.draw())

In [ ]:
ham

In [ ]:
num_qubits = 4
param = 0.3
ham_even, ham_odd = HXXZ(num_qubits, param)

print(ham_even)

circuit = tq.gates.Trotterized(generators=[ham_even], angles=[1.0], steps=2)
circuit += tq.gates.Trotterized(generators=[ham_odd], angles=[1.0], steps=2)

tq.draw(circuit)

openqasmcode = tq.export_open_qasm(circuit)

c = models.Circuit.from_qasm(openqasmcode)

print(c.draw())
# print(c.get_parameters()) # parameters from the RX, RY, SZ gates

In [ ]:
print(c.summary())

In [ ]:
generator = tq.paulis.X(0) * tq.paulis.Y(1)
circuit5 = tq.gates.Trotterized(
    generators=[generator], angles=[1.0], steps=1
)  # acts the same as circuit3

tq.draw(circuit5)